In [1]:
import os
import shutil
import sys
import time
from PIL import Image
from datetime import date, datetime, timedelta

sys.path.insert(0, '..')
from utils import mongodb
from ncm_unico import cursor_ncm_unico, extract_to

## Generate vazios database

#### Necessário acesso ao Servidor ajna.labin.rf08.srf (e habilitação no AJNA) ou acesso ao Servidor mongo.labin.rf08.srf (e senha do banco)

In [ ]:
#if os.path.exists('ncmsunicos_cropped'):
#    shutil.rmtree('ncmsunicos_cropped')

In [ ]:
start = datetime(2017, 7, 1)
end = datetime.today()
limit = 1000
limitportipo = 20
for label in (True, False):
    s0 = time.time()
    cursor = cursor_vazio_nvazio(mongodb, start, end, limit, label, crop=True)
    count = extract_to(cursor, crop=True)
    s1 = time.time()
    print('{:0.2f} segundos para processar {:d} registros'.format((s1 - s0), count))


### Exportar pré processadas para visualização e cópia

#### Esta parte é para gerar arquivos em tamanho menor, possibilitando a cópia e compartilhamento (PenDrive, GoogleDrive, ReceitaDrive)

In [ ]:
if not os.path.exists('ncmsunicos_cropped'):
    os.mkdir('ncmsunicos_cropped')

train_datagen = ImageDataGenerator(
    rescale=1./255,
)

In [ ]:
for classe in ():
    for i, batch in enumerate(train_datagen.flow_from_directory(
        'ncmsunicos_cropped',
        target_size=(144, 288),
        batch_size=10, 
        save_to_dir = 'transformed_ncmsunicos_cropped/' + classe,
        classes=[classe])):
        if i > 40:
            break

## Example: Training a very basic convolutional network

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [8]:
model = models.Sequential([
  layers.Conv2D(16, (5, 5), activation='relu',
                padding='same',
                input_shape=(144, 288, 3)),
  layers.MaxPooling2D(pool_size=(2, 4)),
  layers.Dropout(0.25),
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (1, 1), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(512, (1, 1), activation='relu'),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(156, activation='softmax')
 
])

model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 144, 288, 16)      1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 72, 72, 16)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 72, 72, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 64)       

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'ncmsunicos_cropped',
    target_size=(144, 288),
    batch_size=128,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'ncmsunicos_cropped',
    target_size=(144, 288),
    batch_size=60,
    class_mode='categorical',
    subset='validation'
)


Found 576 images belonging to 156 classes.
Found 81 images belonging to 156 classes.


In [10]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                              verbose=1, min_delta=1e-2, mode='min')

model.fit(train_generator, epochs=50,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

Train for 5 steps, validate for 1 steps
Epoch 1/50
5/5 [==============================] - 8s 2s/step - loss: 5.0428 - acc: 0.0069 - val_loss: 5.0247 - val_acc: 0.0333
Epoch 2/50
5/5 [==============================] - 7s 1s/step - loss: 4.9768 - acc: 0.0330 - val_loss: 4.9150 - val_acc: 0.0333
Epoch 3/50
5/5 [==============================] - 7s 1s/step - loss: 4.8907 - acc: 0.0330 - val_loss: 4.8578 - val_acc: 0.0500
Epoch 4/50
5/5 [==============================] - 6s 1s/step - loss: 4.8057 - acc: 0.0365 - val_loss: 4.8012 - val_acc: 0.0667
Epoch 5/50
5/5 [==============================] - 6s 1s/step - loss: 4.7191 - acc: 0.0521 - val_loss: 4.6743 - val_acc: 0.1000
Epoch 6/50
5/5 [==============================] - 6s 1s/step - loss: 4.5979 - acc: 0.0538 - val_loss: 4.5767 - val_acc: 0.1333
Epoch 7/50
5/5 [==============================] - 6s 1s/step - loss: 4.5676 - acc: 0.0660 - val_loss: 4.6248 - val_acc: 0.1333
Epoch 8/50
5/5 [==============================] - 6s 1s/step - loss: 4.

In [11]:
model.save_weights('gerabasesncmsunicos_cropped.hdf5')

In [ ]:
#Fine tunning
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:-7]:
    layer.trainable = False

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4,
                          verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.Adam(lr=0.000001), loss='binary_crossentropy', metrics=['acc'])
for l in model.layers:
    print(l.name, l.trainable)

In [ ]:
model.fit_generator(train_generator, epochs=20,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

In [ ]:
model.save_weights('gerabasesncmsunicos_cropped_tunned.hdf5')